In [1]:
#più canali
#valutare se settare i margini dei firing rates tra 0.16-2Hz come nel paper
#derivata seconda su waveform papers
# un file con bl e stimulato (clustering su tutto e divisione in due classi in base a indice)

In [2]:
from NewLibraryENG import *
from tqdm.notebook import tqdm
import sklearn.preprocessing as ps
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from random import randint
from fastdtw import fastdtw
import copy
import pymc as pm
#name_data = '2018-11-27T10-29-42POP 2 BL .h5'
#name_data = '2018-11-27T12-03-02POP 2 stimulation.h5'
#name_data='2018-11-27T10-56-39MiP5 KA stimulation.h5'
#name_data='2018-11-27T11-24-28MiP3 stimulation.h5'
#name_data='2018-11-27T10-40-53POP 3 BL .h5'
#name_data='2019-01-23T11-05-52MIP3 health cortical .h5'
#name_data='2019-01-23T15-49-43MEA2 healthy cortical .h5'
#name_data='2019-01-23T16-22-47Pop3 healthy cortical .h5'
#name_data='2019-01-23T16-06-32Pop1 healthy cortical .h5'
name_data='2019-01-24T16-00-33Pop1 24hour after.h5'

complete_string='/Users/Gaia_1/Desktop/allh5files/healthy/healthy 24hrs later/'+name_data

In [3]:
data = h5py.File(complete_string,'r')

data_readings = data['Data']['Recording_0']['AnalogStream']['Stream_0']['ChannelData'][()]
info = data['Data']['Recording_0']['AnalogStream']['Stream_0']['InfoChannel'][()]
info_table = pd.DataFrame(info, columns = list(info.dtype.fields.keys()))
labels = info_table['Label']
readings = pd.DataFrame(data = data_readings.transpose(), columns = labels)
fs = 10000 #Sampling Frequency
print(readings.shape)

(4228000, 60)


In [4]:
inizio=0
#len_data=810000 #192 secondi
len_data=len(readings)
prova=readings.iloc[inizio:len_data, 10:27]
prova=prova.drop([b'Ref'],axis=1)
ref=readings[b'Ref']
ref=ref[inizio:len_data]

print(prova.shape,ref.shape)

(4228000, 16) (4228000,)


In [5]:
prova_rows = range(prova.shape[0])
filt_prova = pd.DataFrame(data = 0, columns=prova.columns, index=prova_rows, dtype = "float32")
lowcut = 300
highcut = 3000
fs=10000
order=8
b,a=butter_bandpass(lowcut,highcut,fs,order=order)
filt_ref=filtfilt(b,a,ref)
#ref_df = pd.DataFrame({b'Ref': filt_ref})
for x in tqdm(range(prova.shape[1])):
    filt_prova.values[:,x] = scipy.signal.filtfilt(b, a, prova.values[:,x])
for electrode in prova.columns:
    filt_prova[electrode] = filt_prova[electrode] - filt_ref
#filt_prova = pd.concat([filt_prova, ref_df], axis=1)
prova=filt_prova
prova.shape

  0%|          | 0/16 [00:00<?, ?it/s]

(4228000, 16)

# Spike detection

In [14]:
threshold=[]
for i,electrode in tqdm(enumerate(prova.columns)):
    threshold.append(4*(scipy.stats.median_abs_deviation(prova[electrode].values)))
    

0it [00:00, ?it/s]

In [18]:
threshold[0]

111.1728190488953

In [23]:
pos_ind=[]
neg_ind=[]
for i,electrode in tqdm(enumerate(prova.columns)):
    channel=prova[electrode]
    thresh=threshold[i]
    pos,neg=new_find_all_spikes(channel,thresh)
    pos_ind.append(pos)
    neg_ind.append(neg)

0it [00:00, ?it/s]

positive spikes 37132 negative spikes 2592 detected spikes: 39724 firing rate:  93.95458845789972
positive spikes 37560 negative spikes 2506 detected spikes: 40066 firing rate:  94.76348155156103
positive spikes 36325 negative spikes 2711 detected spikes: 39036 firing rate:  92.32734153263955
positive spikes 36347 negative spikes 2669 detected spikes: 39016 firing rate:  92.28003784295176
positive spikes 33056 negative spikes 3533 detected spikes: 36589 firing rate:  86.53973509933775
positive spikes 34767 negative spikes 2951 detected spikes: 37718 firing rate:  89.2100283822138
positive spikes 35663 negative spikes 2764 detected spikes: 38427 firing rate:  90.88694418164617
positive spikes 34637 negative spikes 2953 detected spikes: 37590 firing rate:  88.90728476821192
positive spikes 36057 negative spikes 2704 detected spikes: 38761 firing rate:  91.67691579943235
positive spikes 34217 negative spikes 3255 detected spikes: 37472 firing rate:  88.62819299905392
positive spikes 36021

pos_ind=[]
neg_ind=[]
for electrode in tqdm(prova.columns):
    channel=prova[electrode]
    pos, neg=find_all_spikes(channel)
    pos_ind.append(pos)
    neg_ind.append(neg)

BL: window 3000 (threshold 4 MAD) firing 18    
BL: window 300 (thresh 3), firing 89    
KA: window 3000 (thresh 3), firing 94    
KA: window 3000 (thresh 4), firing 29    
KA: window 300, (thresh 4) firing 39   
KA: window 300, (thresh 3) firing 108   

plt.figure(figsize=(30, 15))
plt.plot(prova, label='Signal Data')
plt.scatter(pos, [prova[i] for i in pos], c='red', marker='o', label='Local Maxima')
plt.scatter(neg, [prova[i] for i in neg], c='green', marker='o', label='Local Minima')
plt.xlabel('Sample Index')
plt.ylabel('Signal Value')
plt.legend()
plt.grid(True)
plt.title('Signal Data with Detected Spikes')
#plt.axis([0,3000,-5,5])
#plt.savefig('spikes1.jpg')
plt.show()

minima,maxima=RMM(prova)
print(len(maxima))
#segnale BASELINE: firing rate=70 (spikes 26148)
#segnale Stimulation: firing rate=69 (spikes 32980)
#segnale KA stimulation: firing rate=81 (spikes 47860)

minima,maxima=find_spikes(prova)
len(maxima)
#segnale BASELINE: firing rate=27 (spikes 10276)
#segnale Stimulation: firing rate=27 (spikes 13198)
#segnale KA stimulation: firing rate=32 (spikes 19178)

minima,maxima=find_spikes_with_memory(prova)
len(maxima)
#segnale BASELINE: firing rate=25 (spikes 9270)
#segnale Stimulation: firing rate=22 (spikes 10579)
#segnale KA stimulation: firing rate=8 (spikes 5054)

plt.figure(figsize=(30, 15))
plt.plot(prova, label='Signal Data')
plt.scatter(minima, [prova[i] for i in minima], c='red', marker='o', label='Local Minima')
plt.scatter(maxima, [prova[i] for i in maxima], c='green', marker='x', label='Local maxima')
plt.xlabel('Sample Index')
plt.ylabel('Signal Value')
plt.legend()
plt.grid(True)
plt.title('Signal Data with Detected Spikes')
#plt.axis([0,30000,-5,5])
#plt.savefig('spikes2.jpg')
plt.show()

# Cutouts

In [ ]:
pos_cut=[]
neg_cut=[]
n_pos=[]
n_neg=[]

for i,electrode in enumerate(tqdm(prova.columns)):
    pos=pos_ind[i]
    neg=neg_ind[i]
    channel=prova[electrode]
    pos_cut1,n_pos1, neg_cut1,n_neg1 = cut(pos,neg,channel)
    pos_cut.append(pos_cut1)
    neg_cut.append(neg_cut1)
    n_pos.append(n_pos1)
    n_neg.append(n_neg1)
#savedp = copy.deepcopy(pos_cut)
#savedn = copy.deepcopy(neg_cut)

In [ ]:
for i in (tqdm(range(len(pos_cut)))):
    pos_cut[i]=mask_cuts(pos_cut[i])
    neg_cut[i]= mask_cuts(neg_cut[i])

x=randint(1,len(pos_cut)-1)
y=randint(1,len(pos_cut[x])-1)

plt.plot(savedp[x][y])
plt.plot(pos_cut[x][y])
plt.show()
#plt.plot(savedn[x])
#plt.plot(neg_cut[x])
#plt.show()

import pandas as pd
pca = PCA()
pca.fit(pos_cut)
explained_variances = pca.explained_variance_ratio_
explained_variance_df = pd.DataFrame(data={'Explained Variance': explained_variances},
                                     index=range(1, len(explained_variances) + 1))
explained_variance_df.transpose()

cumulative_explained_variance = np.cumsum(explained_variances)
cumulative_explained_variance_df = pd.DataFrame(data={'Cumulative Explained Variance': cumulative_explained_variance},
                                                index=range(1, len(cumulative_explained_variance) + 1))
cumulative_explained_variance_df.transpose()


# Clustering

final_data_pos=[]
for channel in (tqdm(range(len(pos_cut)))):
    channel_clusters= hdbscan_clustering(pos_cut[channel],n_pos[channel],prova.iloc[:,channel])
    final_data_pos.append(channel_clusters)
    

BL (window 3000, thresh 4) (less function) fuzzy (1.8, 4, 3.99)   
BL (window 300) (less function) fuzzy (19, 18, 9)   
BL (window 300) (less function) dbscan (30, 9)   
BL (window 300) (bit less function std) fuzzy (8.9, 18.2, 17.8)   
BL (window 300) (more cut function) fuzzy (8.9, 17.7, 18.3)   
BL (window 300) (more cut function) dbscan (29, 6)   
BL (less cut function) fuzzy: (15, 8 ,15)   

KA (window 300, thresh 4) (less function) fuzzy (3.8, 4, 1.9)   
KA (window 3000, thresh 4) (less function) fuzzy (2.6, 1.2, 2.8)   
KA stimualtion 3 clusters fuzzy: (13, 6, 12)   
KA stim (window 300) (new cut function) (1.17,2.55,2.37)   

firing rate between 0.16 and 2 Hz
fr=len(cluster)*10000/len(alldata)=0.16-:2
len(cluster)=len(spike_list)/n_clusters
len(spike_list)*10.000/n_clusters*len(alldata)=0.16-2
n_min=10.000*len(spike_list)/2*len(alldata)
n_max=10.000*len(spike_list)/0.16*len(alldata)


final_data_neg=[]
for channel in (tqdm(range(len(neg_cut)))):
    channel_clusters= hdbscan_clustering(neg_cut[channel],n_neg[channel],data)
    final_data_neg.append(channel_clusters)

In [ ]:
# togliere dal best score il DB score e usare solo il silhouette score ma su 15 prove
# fare scrematura una volta formati i clusters, togliendo gli spike con distanza DTW sopra soglia
# provare anche DTW tra medie di cluster

In [ ]:
final_data_pos=[]
for channel in (tqdm(range(len(pos_cut)))):
    channel_clusters=nested_clus(pos_cut[channel],'fuzzy',n_pos[channel],prova.iloc[:,channel])
    final_data_pos.append(channel_clusters)

In [ ]:
final_data_neg=[]
print(name_data)
for channel in (tqdm(range(len(neg_cut)))):
    channel_clusters=nested_clus(neg_cut[channel],'fuzzy',n_neg[channel],prova.iloc[:,channel])
    final_data_neg.append(channel_clusters)

BL (less cut function) (window 3000, thresh4) fuzzy: (3.5, 3.5, 1.6) silhouette: 0.231    
BL (less cut function) (window 300) fuzzy: (17, 15, 6) silhouette: 0.215    
BL (more cut function) (window 300) fuzzy: (15, 16, 7) silhouette: 0.218    
BL (less cut function) fuzzy: (13, 13, 6) silhouette: 0.226    

KA (less cut function) (window 300, thresh4) fuzzy: (1.9, 1.12, 1.9) silhouette: 0.229   
KA (less cut function) (window 3000, thresh4) fuzzy: (1.4, 0.89, 1.4) silhouette: 0.223    
KA stimulation 3 clusters fuzzy: (3.9, 3.7, 2) silhouette: 0.214    
KA stimulation (more cut function) (window 3000) fuzzy: (2, 3.9, 3.7) silhouette: 0.214    
KA stimulation (more cut function) (window 300) fuzzy (4.5, 4.4, 2.2) silhouette: 0.204    
KA stimulation (more cut function) (window 300) dbscan (10.8,10.8)    

# Save

In [ ]:
len(final_data_pos[0][0])

In [ ]:
final_data_neg

In [ ]:
neurons=[]
#min_len=len(final_data_pos[0][0])
for electrode in final_data_pos:
    for neuron in electrode:
        #if len(neuron)>1000:
        #min_len=min(min_len,len(neuron))
        neurons.append(neuron)
        #print(len(neuron))
print('positive neurons:',len(neurons))


In [ ]:
for electrode in final_data_neg:
    for neuron in electrode:
        #if len(neuron)>1000:
        neurons.append(neuron)
        min_len=min(min_len,len(neuron))
        print(len(neuron))
print('negative neurons: ',len(neurons))

In [ ]:
len(neurons[21])

In [ ]:
adj_neur=[]
for neuron in neurons:
    neu=neuron#[0:min_len]
    adj_neur.append(neu)
print(min_len,len(adj_neur))

neurons = []
lens=[]
for electrode in final_data_pos:
    for neuron in electrode:
        if len(neuron)>1000:
            lens.append(len(neuron))
for neuron in final_data_neg:
    for neuron in electrode:
        if len(neuron)>1000:
            lens.append(len(neuron))

avg_len=int(np.mean(lens))
for electrode in final_data_pos:
    for neuron in electrode:
        if len(neuron)>1000:
            print(len(neuron))
            diff = avg_len-len(neuron)
            if diff<0:
                neurons.append(neuron[0:avg_len])
            else:
                neurons.append(np.concatenate((neuron,np.zeros([diff]))))
print(len(neurons),avg_len)

for neuron in final_data_neg:
    for neuron in electrode:
        if len(neuron)>1000:
            print(len(neuron))
            diff = avg_len-len(neuron)
            if diff<0:
                neurons.append(neuron[0:avg_len])
            else:
                neurons.append(np.concatenate((neuron,np.zeros([diff]))))
print(len(neurons),avg_len)

adj_neur=[]
counter = 0
max_len=0

for neu in neurons:
    print('counter: ',counter,neu.shape[0])
    if neu.shape[0]>max_len:
        max_len=neu.shape[0]
    counter+=1
for neuron in neurons:
    if neuron.shape[0]<=max_len:
        diff = max_len-neuron.shape[0]
        adj_neur.append(np.concatenate((neuron,np.zeros([diff]))))
print(max_len)

In [ ]:
save_data = 'After'+name_data+'.txt'
print(name_data)

In [ ]:
np.savetxt("/Users/Gaia_1/Desktop/tesi/Data after SS/%s.txt" % save_data,adj_neur, delimiter=', ', fmt='%12.8f')

## Point Process

In [ ]:
neurons=[]
for neuron in final_data_pos:
    neurons.append(neuron)
for neuron in final_data_neg:
    neurons.append(neuron)
neurons

In [ ]:
neurons[0]

In [ ]:
x=randint(0,len(neurons))
#x=21
print('neur: ',x)
#if len(neurons[x])>1000:
print('length: ',len(neurons[x]))
data_healthy=neurons[x]
ISI_healthy = np.diff(data_healthy)/10000
#else:
    #print('try again')

In [ ]:
locs_diff=np.diff(data_healthy)
#plt.axis([-3,500,0,0.05])
plt.hist(ISI_healthy, bins=50, density=False, alpha=0.5, color='blue', edgecolor='black')

In [ ]:
map_estimate,ppc_trace = Bayesian_mixture_model(ISI_healthy)

In [ ]:
plt.hist(ISI_healthy, bins=50, density=False, alpha=0.5, color='blue', edgecolor='black')
#plt.plot(hist[0])

In [ ]:
plt.plot(hist[0]/max(hist[0]))

In [ ]:
hist[1][:-1]

In [ ]:
max(ISI_healthy)

In [ ]:
ISI_healthy

In [ ]:
max(ISI_healthy/max(ISI_healthy))

In [ ]:
plt.hist(ISI_healthy/max(ISI_healthy),bins)

In [ ]:
len(bins)

In [ ]:
bins = np.arange(0, .5, 1e-3) 
plt.figure (figsize=(10,4))

hist = np.histogram(ppc_trace['posterior_predictive']['like'].values,bins=bins)
#plt.axis([0,0.3,0,160])
a= plt.hist(ISI_healthy,bins)
plt.plot(hist[1][:-1],hist[0]/1000,linewidth=3);

In [ ]:
lista_samples=[]
for i in list(ppc_trace['posterior_predictive']['like'].values):
    lista_samples.extend(i)

In [ ]:
counts, _ = np.histogram(ISI_healthy, bins) 
prob_emp = counts / np.sum(counts)
counts, _ = np.histogram(lista_samples, bins) 
prob_model = counts / np.sum(counts)

Femp = np.cumsum(prob_emp)           
Fmodel = np.cumsum(prob_model)          
plt.figure()
plt.plot(bins[:-1], Femp)                
plt.plot(bins[:-1], Fmodel, 'r')       
#plt.xlim([0, 0.2])                  
plt.xlabel('Time [s]')
plt.ylabel('CDF')
plt.legend(['Empirical','Model'])
plt.show()

In [ ]:
plt.figure()
Nlow = len(ISI_healthy)  
# Plot the confidence bounds
plt.plot([0, 1], [x + 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'r:')
plt.plot([0, 1], [x - 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'r:')
plt.plot(Femp, Fmodel)
plt.axis([0, 1, 0, 1])         
plt.xlabel('Model CDF')
plt.ylabel('Empirical CDF')
plt.show()

# Multiple channel point process

In [ ]:
dataframe = pd.DataFrame()
list_neurons = neurons
counter=0
print('Original number of neurons: ',len(list_neurons))
net=name_data
for neuron in list_neurons:
    neuron=neuron[neuron>0*10000]
    neuron=neuron[neuron<200*10000]
    print('  Neuron with ',neuron.shape[0],'spikes')
    if neuron.shape[0]>1000:

        counter+=1
    else:
        print('    Excluded neuron with n spikes = ',neuron.shape[0])
        continue

    ISI_healthy = np.diff(neuron)/10000


    map_estimate = Bayesian_mixture_model(ISI_healthy)
    if 'Healthy' in net:
        print('target Healthy')
        map_estimate['Target']=0

    elif 'healthy' in net:
        print('target healthy')
        map_estimate['Target']=0

    elif 'health' in net:
        print('target health')
        map_estimate['Target']=0

    else:
        print('target pathological')
        map_estimate['Target']=1

    if 'KA' in net:
        map_estimate['Stimulation']=1

    elif 'stimulation' in net:
        map_estimate['Stimulation']=1
    else:
        map_estimate['Stimulation']=0

    df = pd.DataFrame.from_dict(map_estimate,orient='index')
    dataframe = pd.concat([dataframe,df],axis = 1)
print('Final number of neurons: ',counter)
print('Target = ',target)

In [ ]:
final = dataframe.T
final.to_csv('Data after PP/DataAfter')

## Classification

In [ ]:
def performance_visualizer(trials_obj,n_models,choice=False,**choice_var):
    
    import pandas as pd
    
    performance = [1-t['result']['loss'] for t in trials_obj.trials]
    
    
    hyperparam= list(trials_obj.trials[0]['misc']['vals'].keys())
    
    values_dict ={}
    
    for i in hyperparam:
        
        values_dict[i]=[]
        
        for j in trials_obj.trials:
            
            if(len(j['misc']['vals'][i])==0):
                
                values_dict[i].append(np.NaN)
                
            else:
            
                values_dict[i].append(j['misc']['vals'][i][0])
                
    out = pd.DataFrame.from_dict(values_dict)
    
    out['performance'] = performance
    
    out=out.sort_values(by=['performance'])
    
    
    if choice:
        
        for i in list(choice_var.keys()):
        
            for j,_ in enumerate(choice_var[i]):
        
                out[i]=out[i].replace(j,choice_var[i][j])
    
    return out.tail(n_models)

In [ ]:
dataset = pd.read_csv('Data after PP/Data')
dataset = dataset.drop(['Unnamed: 0'],axis=1)
dataset